In [1]:
# Phase 1 Integration Test
# This notebook tests that all Phase 1 components work together

import sys
sys.path.append('..')

from src.data.bigquery_loader import BigQueryLoader
from src.data.preprocessing import DataPreprocessor
from src.models.baseline import (
    GlobalAverageBaseline,
    MovieAverageBaseline,
    PopularityBasedRecommender
)
from src.utils.evaluation import rmse, mae

print("=" * 60)
print("PHASE 1 INTEGRATION TEST")
print("=" * 60)

# Step 1: Load Data
print("\n1. Loading data from BigQuery...")
loader = BigQueryLoader()
ratings = loader.load_ratings(limit=10000)
movies = loader.load_movies()
print(f"✓ Loaded {len(ratings):,} ratings and {len(movies):,} movies")

# Step 2: Preprocess
print("\n2. Preprocessing data...")
preprocessor = DataPreprocessor(ratings, movies)
train, test = preprocessor.create_train_test_split(test_size=0.2)
print(f"✓ Split into {len(train):,} train and {len(test):,} test ratings")

# Step 3: Train Baselines
print("\n3. Training baseline models...")

global_model = GlobalAverageBaseline()
global_model.train(train)
print("✓ Global average baseline trained")

movie_model = MovieAverageBaseline()
movie_model.train(train)
print("✓ Movie average baseline trained")

pop_model = PopularityBasedRecommender(min_ratings=20)
pop_model.train(train, movies)
print("✓ Popularity recommender trained")

# Step 4: Evaluate
print("\n4. Evaluating models...")

# Evaluate Global Average
predictions = []
actuals = []
for _, row in test.iterrows():
    pred = global_model.predict(row['userId'], row['movieId'])
    predictions.append(pred)
    actuals.append(row['rating'])

import numpy as np
predictions = np.array(predictions)
actuals = np.array(actuals)

global_rmse = rmse(predictions, actuals)
global_mae = mae(predictions, actuals)
print(f"Global Average - RMSE: {global_rmse:.4f}, MAE: {global_mae:.4f}")

# Evaluate Movie Average
predictions = []
for _, row in test.iterrows():
    pred = movie_model.predict(row['userId'], row['movieId'])
    predictions.append(pred)

predictions = np.array(predictions)

movie_rmse = rmse(predictions, actuals)
movie_mae = mae(predictions, actuals)
print(f"Movie Average  - RMSE: {movie_rmse:.4f}, MAE: {movie_mae:.4f}")

# Step 5: Test Recommendations
print("\n5. Testing recommendations...")

top_movies = pop_model.get_top_n_recommendations(n=5)
print("\nTop 5 Popular Movies:")
print(top_movies[['title', 'popularity_score', 'num_ratings']])

action_movies = pop_model.get_top_n_recommendations(n=3, genre='Action')
print("\nTop 3 Action Movies:")
print(action_movies[['title', 'genres', 'popularity_score']])

print("\n" + "=" * 60)
print("✓ ALL INTEGRATION TESTS PASSED!")
print("=" * 60)

print("\nPhase 1 Summary:")
print("- Data pipeline: Working ✓")
print("- Preprocessing: Working ✓")
print("- Baseline models: Working ✓")
print("- Evaluation: Working ✓")
print("- Popularity recommender: Working ✓")
print("\n→ Ready for Phase 2: SVD Model Development")

PHASE 1 INTEGRATION TEST

1. Loading data from BigQuery...
✓ Loaded 10,000 ratings and 10,329 movies

2. Preprocessing data...
✓ Split into 8,000 train and 2,000 test ratings

3. Training baseline models...
✓ Global average baseline trained
✓ Movie average baseline trained
✓ Popularity recommender trained

4. Evaluating models...
Global Average - RMSE: 0.5469, MAE: 0.4954
Movie Average  - RMSE: 0.6052, MAE: 0.5008

5. Testing recommendations...

Top 5 Popular Movies:
                                              title  popularity_score  \
5  Star Wars: Episode I - The Phantom Menace (1999)          1.520610   
4                              Mars Attacks! (1996)          1.508415   
1                                 Waterworld (1995)          1.496220   
7                   Blair Witch Project, The (1999)          1.391744   
2            Dumb & Dumber (Dumb and Dumber) (1994)          1.379681   

   num_ratings  
5           21  
4           21  
1           21  
7           23  
2   